In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
ls

gdrive/  sample_data/


In [0]:
cd gdrive/My Drive/Datasets/cat_vs_dog/

/content/gdrive/My Drive/Datasets/cat_vs_dog


In [0]:
!pip install tensorflow==2.*
!pip install tensorflow-hub>=0.6.0
!pip install tf-nightly

In [0]:
!pip uninstall tensorflow_estimator
!pip install tensorflow_estimator

In [0]:
import tensorflow as tf
import numpy as np
import tensorflow_hub as hub
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

In [0]:
classifier = Sequential()

classifier.add(Convolution2D(32, 3, 3, input_shape = (64, 64, 3), activation = 'relu'))

classifier.add(MaxPooling2D(pool_size = (2, 2)))

classifier.add(Convolution2D(32, 3, 3, activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

classifier.add(Flatten())

classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

valid_gen = test_datagen.flow_from_directory('test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')


Found 8009 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [0]:
classifier.fit_generator(valid_gen,epochs=2)

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
Train for 63 steps
Epoch 1/2
63/63 [==============================] - 8s 121ms/step - loss: 0.6926 - accuracy: 0.5085
Epoch 2/2
63/63 [==============================] - 6s 102ms/step - loss: 0.6875 - accuracy: 0.5445


**TFlite conversion**

In [0]:
!mkdir "tflite_models"
TFLITE_MODEL = "tflite_models/catvsdog.tflite"
TFLITE_WEIGHT_QUANT_MODEL = "tflite_models/catvsdog_weight_quant.tflite"
TFLITE_FP16_QUANT_MODEL="tflite_models/catvsdog_fp16_quant.tflite"
TFLITE_INTEGER_QUANT_MODEL="tflite_models/catvsdog_integer_quant.tflite"

mkdir: cannot create directory ‘tflite_models’: File exists


In [0]:
CATVSDOG_SAVED_MODEL = "saved_models/catvsdog"
tf.saved_model.save(classifier, CATVSDOG_SAVED_MODEL)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: saved_models/catvsdog/assets


In [0]:
catvsdog_model = hub.load(CATVSDOG_SAVED_MODEL)
print(catvsdog_model)

<tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject object at 0x7f1d67dcefd0>


**0) Simple Conversion to tflite without quantization**

In [0]:
from tensorflow import lite
# Converting a tf.Keras model to a TensorFlow Lite model.
converter = lite.TFLiteConverter.from_keras_model(classifier)
tflite_model = converter.convert()
open(TFLITE_MODEL, "wb").write(tflite_model)

60376

**1) Weight Quantization**

In [0]:
from tensorflow import lite
converter = lite.TFLiteConverter.from_keras_model(classifier)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_weight_quant_model = converter.convert()
open(TFLITE_WEIGHT_QUANT_MODEL, "wb").write(tflite_weight_quant_model)

20480

**2) Full Integer Quantization**

In [0]:
import tensorflow as tf
num_calibration_steps=50
from matplotlib.image import imread
def representative_dataset_gen():
  for _ in range(num_calibration_steps):
    name="training_set/dogs/dog."+str(_+1)+".jpg"
    image = imread(name)
    image = np.resize(image,(64,64,3))
    l=[]
    l.append(image)
    image=np.array(l)
    image =image.astype(np.float32)
    yield [image]
  for _ in range(num_calibration_steps):
    name="training_set/cats/cat."+str(_+1)+".jpg"
    image = imread(name)
    image = np.resize(image,(64,64,3))
    l=[]
    l.append(image)
    image=np.array(l)
    image =image.astype(np.float32)
    yield [image]

converter = tf.lite.TFLiteConverter.from_keras_model(classifier)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = tf.lite.RepresentativeDataset(representative_dataset_gen) 
tflite_integer_quant_model = converter.convert()

open(TFLITE_INTEGER_QUANT_MODEL, "wb").write(tflite_integer_quant_model)

20416

**Float16 Quantization**

In [0]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_keras_model(classifier)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_fp16_quant_model = converter.convert()
open(TFLITE_FP16_QUANT_MODEL, "wb").write(tflite_fp16_quant_model)

32332

**Load TFLite model and see some details about input/output**

In [0]:
tflite_interpreter = tf.lite.Interpreter(model_path=TFLITE_FP16_QUANT_MODEL)

input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

== Input details ==
name: conv2d_input
shape: [ 1 64 64  3]
type: <class 'numpy.float32'>

== Output details ==
name: Identity
shape: [1 1]
type: <class 'numpy.float32'>


**Get images and labels batch from validation dataset generator**

In [0]:
import numpy as np
val_image_batch, val_label_batch = next(iter(valid_gen))
true_label_ids = np.argmax(val_label_batch, axis=-1)

print("Validation batch shape:", val_image_batch.shape)

Validation batch shape: (32, 64, 64, 3)


**Loading models, reshaping test images and then running prediction in TFlite-quantised model**

In [0]:
# Load quantized TFLite model
tflite_interpreter_quant = tf.lite.Interpreter(model_path=TFLITE_FP16_QUANT_MODEL)

# Learn about its input and output details
input_details = tflite_interpreter_quant.get_input_details()
output_details = tflite_interpreter_quant.get_output_details()

# Resize input and output tensors to handle batch of 32 images
tflite_interpreter_quant.resize_tensor_input(input_details[0]['index'], (32, 64, 64, 3))
tflite_interpreter_quant.resize_tensor_input(output_details[0]['index'], (32, 5))
tflite_interpreter_quant.allocate_tensors()

input_details = tflite_interpreter_quant.get_input_details()
output_details = tflite_interpreter_quant.get_output_details()

print("== Input details ==")
print("name:", input_details[0]['name'])
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])

print("\n== Output details ==")
print("name:", output_details[0]['name'])
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

# Run inference
tflite_interpreter_quant.set_tensor(input_details[0]['index'], val_image_batch)

tflite_interpreter_quant.invoke()

tflite_q_model_predictions = tflite_interpreter_quant.get_tensor(output_details[0]['index'])
print("\nPrediction results shape:", tflite_q_model_predictions.shape)

== Input details ==
name: conv2d_input
shape: [32 64 64  3]
type: <class 'numpy.float32'>

== Output details ==
name: Identity
shape: [32  1]
type: <class 'numpy.float32'>

Prediction results shape: (32, 1)


In [0]:
print(tflite_q_model_predictions)

[[0.4496088 ]
 [0.4463991 ]
 [0.4271283 ]
 [0.5131845 ]
 [0.5050647 ]
 [0.48915556]
 [0.4704319 ]
 [0.48002702]
 [0.4833661 ]
 [0.46815324]
 [0.45691365]
 [0.46552303]
 [0.45869052]
 [0.46291965]
 [0.4834959 ]
 [0.57243824]
 [0.44449946]
 [0.4601686 ]
 [0.43426186]
 [0.44600034]
 [0.50837713]
 [0.49113706]
 [0.4622544 ]
 [0.45050168]
 [0.5157333 ]
 [0.45288408]
 [0.44457397]
 [0.48940024]
 [0.46762773]
 [0.442197  ]
 [0.50544566]
 [0.48047516]]
